In [235]:
import selenium
import requests
from bs4 import BeautifulSoup

In [236]:
URL = "https://www.latamairlines.com/pe/es/ofertas-vuelos?origin=LIM&inbound=2023-02-28T17%3A00%3A00.000Z&outbound=2022-12-18T17%3A00%3A00.000Z&destination=MAD&adt=1&chd=0&inf=0&trip=RT&cabin=Economy&redemption=false&sort=RECOMMENDED"

In [237]:
from selenium import webdriver

In [238]:
agent = {"User-Agent":"Mozilla/5.0"}
r = requests.get(URL, agent)
s = BeautifulSoup(r.text, 'lxml')

In [239]:
# options = webdriver.FirefoxOptions()
# options.add_argument('--incognito')
driver = webdriver.Firefox(executable_path='driver/geckodriver.exe')

In [240]:
driver.get(URL)

In [241]:
vuelos = driver.find_elements_by_xpath('//ol[@aria-label="Vuelos disponibles."]//li[@class="sc-bJTOcE dnJSKm"]')
vuelo = vuelos[2]

In [297]:
from time import sleep

In [298]:
def getInfoVuelo(vuelo, delay=1):
    precios = getPrecios(vuelo)
    button_itinerary = vuelo.find_element_by_xpath('.//a[@class="sc-iwsKbI xkrrd"]')
    button_itinerary.click()
    sleep(delay)
    segments = vuelo.find_elements_by_xpath("//div[contains(@class, 'path-info')]/parent::section")
    segments = getSegmentos(segments)
    close_itinerary = driver.find_element_by_xpath('//div[@class="MuiDialog-container MuiDialog-scrollPaper"]')
    close_itinerary.click()
    sleep(delay)
    tiempo = vuelo.find_elements_by_xpath('.//div[contains(@class, "sc-ekHBYt")]/div[contains(@class, "flight-information")]/span[@class="sc-eTyWNx ilRvHO"]')
    duracion = vuelo.find_element_by_xpath('.//div[contains(@class, "sc-ekHBYt")]/div[contains(@class, "flight-duration")]/span[@class="sc-bsVVwV jAgyHF"]').text
    salida = tiempo[0].text
    llegada = tiempo[1].text.replace("\n", " ")
    return { 'Tiempo': {'Salida': salida, 'Llegada': llegada, 'Duración': duracion},
             'Precios': precios,
             'Segmentos': segments}

In [306]:
getInfoVuelo(vuelos[0], 0.6)

{'Tiempo': {'Salida': '21:10',
  'Llegada': '14:30 +1',
  'Duración': '11 h 20 min'},
 'Precios': [('877.25', 'USD', 'plus'),
  ('1033.01', 'USD', 'top'),
  ('1859.01', 'USD', 'top')],
 'Segmentos': [{'salida': {'origen': 'LIM',
    'tiempo': '21:10',
    'aeropuerto': 'J Chavez Intl.'},
   'llegada': {'origen': 'MAD',
    'tiempo': '14:30',
    'aeropuerto': 'Barajas Intl.'},
   'avion': 'Airbus A350-900',
   'duracion': '11 h 20 min'}]}

In [242]:
tiempo = vuelo.find_elements_by_xpath('.//div[contains(@class, "sc-ekHBYt")]/div[contains(@class, "flight-information")]/span[@class="sc-eTyWNx ilRvHO"]')
duracion = vuelo.find_element_by_xpath('.//div[contains(@class, "sc-ekHBYt")]/div[contains(@class, "flight-duration")]/span[@class="sc-bsVVwV jAgyHF"]').text

In [243]:
salida = tiempo[0].text
llegada = tiempo[1].text.replace("\n", " ")
print("Salida: {}\nLlegada: {}\nDuración: {}".format(salida, llegada, duracion))

Salida: 6:55
Llegada: 6:00 +1
Duración: 17 h 5 min


In [244]:
button_itinerary = vuelo.find_element_by_xpath('.//a[@class="sc-iwsKbI xkrrd"]')

In [245]:
button_itinerary.click()

In [246]:
segments = vuelo.find_elements_by_xpath("//div[contains(@class, 'path-info')]/parent::section")

In [291]:
def getSegmentos(segmentos):
    a_list = [getSecciones(segmento) for segmento in segmentos]
    return a_list

In [293]:
getSegmentos(segments)

[{'salida': {'origen': 'LIM',
   'tiempo': '6:55',
   'aeropuerto': 'J Chavez Intl.'},
  'llegada': {'origen': 'SCL',
   'tiempo': '11:30',
   'aeropuerto': 'A. Merino Benitez Intl.'},
  'avion': 'Airbus A320',
  'duracion': '2 h 35 min'},
 {'salida': {'origen': 'SCL',
   'tiempo': '13:20',
   'aeropuerto': 'A. Merino Benitez Intl.'},
  'llegada': {'origen': 'MAD',
   'tiempo': '6:00',
   'aeropuerto': 'Barajas Intl.'},
  'avion': 'Airbus A350-900',
  'duracion': '12 h 40 min'}]

In [247]:
segment = segments[0]

In [248]:
avion = segment.find_element_by_xpath('.//span[@class="airplane-code"]').text

In [249]:
salida = segment.find_element_by_xpath(".//div[contains(@class,'sc-ldcLGC')]")
llegada = segment.find_element_by_xpath(".//div[contains(@class,'sc-hTQSVH')]")

In [250]:
origen = salida.find_element_by_xpath('./div[@class="iataCode"]/span').text
tiempo = salida.find_element_by_xpath('./div[@class="iataCode"]/span[@class="time"]').text
aeropuerto = salida.find_element_by_xpath('.//span[@class="ariport-name"]').text

In [289]:
def getSecciones(segment):
    salida = segment.find_element_by_xpath(".//div[contains(@class,'sc-ldcLGC')]")
    llegada = segment.find_element_by_xpath(".//div[contains(@class,'sc-hTQSVH')]")
    avion = segment.find_element_by_xpath('.//span[@class="airplane-code"]').text
    duration = segment.find_element_by_xpath('.//span[contains(@class, "pathInfo-duration")]/parent::*/span[@class="time"]').text
    return {'salida': getDatos(salida), 'llegada': getDatos(llegada), 'avion': avion, 'duracion': duration}

In [290]:
getSecciones(segments[1])

{'salida': {'origen': 'SCL',
  'tiempo': '13:20',
  'aeropuerto': 'A. Merino Benitez Intl.'},
 'llegada': {'origen': 'MAD', 'tiempo': '6:00', 'aeropuerto': 'Barajas Intl.'},
 'avion': 'Airbus A350-900',
 'duracion': '12 h 40 min'}

In [261]:
def getDatos(seccion):
    origen = seccion.find_element_by_xpath('./div[@class="iataCode"]/span').text
    tiempo = seccion.find_element_by_xpath('./div[@class="iataCode"]/span[@class="time"]').text
    aeropuerto = seccion.find_element_by_xpath('.//span[@class="ariport-name"]').text
    return {'origen': origen, 'tiempo': tiempo, 'aeropuerto': aeropuerto}

In [233]:
driver.find_element_by_xpath('//div[@class="MuiDialog-container MuiDialog-scrollPaper"]').click()

In [196]:
vuelo.click()

In [197]:
tabla_tarifas = driver.find_element_by_xpath('//ol[@aria-label="Tarifas disponibles."]')

In [198]:
monedas = tabla_tarifas.find_elements_by_xpath('.//span[contains(@class, "currency")]')
tarifas = tabla_tarifas.find_elements_by_xpath('.//span[@class="sc-ccvjgv ddOTqA"]')
tarifas = list(map(lambda x: x.text, tarifas))
monedas = list(map(lambda x: x.text, monedas))
dict(zip(tarifas, monedas))

{'1493.21': 'USD', '1694.99': 'USD', '4490.41': 'USD'}

In [209]:
def getPrecios(vuelo):
    '''
    Función que retorna los precios de un vuelo con el siguiente orden (tarifas, monedas, tiers).
    '''
    vuelo.click()
    tabla_tarifas = driver.find_element_by_xpath('//ol[@aria-label="Tarifas disponibles."]')
    monedas = tabla_tarifas.find_elements_by_xpath('.//span[contains(@class, "currency")]')
    tarifas = tabla_tarifas.find_elements_by_xpath('.//span[@class="sc-ccvjgv ddOTqA"]')
    tier = tabla_tarifas.find_elements_by_xpath('.//span[@class="sc-igaqVs kvfVRk"]')
    tarifas = list(map(lambda x: x.text, tarifas))
    monedas = list(map(lambda x: x.text, monedas))
    tier = list(map(lambda x: x.text, tier))
    button = vuelo.find_element_by_xpath('.//button[contains(@class,"MuiButton-disableElevation")]')
    button.click()
    return list(zip(tarifas, monedas, tier))

In [200]:
vuelos[0].click()

In [210]:
getPrecios(vuelos[23])

[('1092.01', 'USD', 'plus'),
 ('1276.09', 'USD', 'top'),
 ('3219.55', 'USD', 'top')]

In [ ]:
driver.close()